In [23]:
using CSV
using DataFrames
using Glob
using DBFTables
using TableOperations
using Geodesy
using Tables
d = CSV.read("metro_grid.csv", DataFrame)

,lon,lat
,Float64,Float64
1,-94.011,44.4714
2,-94.002,44.4714
3,-93.993,44.4714
4,-94.011,44.4778
5,-93.9751,44.4714
6,-93.9841,44.4714
7,-93.993,44.4778
8,-94.002,44.4778
9,-94.011,44.4842


In [4]:
files = glob("../webapp/modules/geotools/geotools/data/new_parcels/*.dbf")

7-element Array{String,1}:
 "../webapp/modules/geotools/geotools/data/new_parcels/ParcelsAnokaPoints.dbf"
 "../webapp/modules/geotools/geotools/data/new_parcels/ParcelsCarverPoints.dbf"
 "../webapp/modules/geotools/geotools/data/new_parcels/ParcelsDakotaPoints.dbf"
 "../webapp/modules/geotools/geotools/data/new_parcels/ParcelsHennepinPoints.dbf"
 "../webapp/modules/geotools/geotools/data/new_parcels/ParcelsRamseyPoints.dbf"
 "../webapp/modules/geotools/geotools/data/new_parcels/ParcelsScottPoints.dbf"
 "../webapp/modules/geotools/geotools/data/new_parcels/ParcelsWashingtonPoints.dbf"

In [123]:
DBFData = DBFTables.Table(files[1]) |> Tables.rowtable

2-element Array{NamedTuple{(:COUNTY_PIN, :STATE_PIN, :ANUMBERPRE, :ANUMBER, :ANUMBERSUF, :ST_PRE_MOD, :ST_PRE_DIR, :ST_PRE_TYP, :ST_PRE_SEP, :ST_NAME, :ST_POS_TYP, :ST_POS_DIR, :ST_POS_MOD, :SUB_TYPE1, :SUB_ID1, :SUB_TYPE2, :SUB_ID2, :ZIP, :ZIP4, :CTU_NAME, :CTU_ID_TXT, :POSTCOMM, :CO_CODE, :CO_NAME, :STATE_CODE, :LOT, :BLOCK, :PLAT_NAME, :OWNER_NAME, :OWNER_MORE, :OWN_ADD_L1, :OWN_ADD_L2, :OWN_ADD_L3, :OWN_ADD_L4, :TAX_NAME, :TAX_ADD_L1, :TAX_ADD_L2, :TAX_ADD_L3, :TAX_ADD_L4, :LANDMARK, :HOMESTEAD, :ACRES_POLY, :ACRES_DEED, :EMV_LAND, :EMV_BLDG, :EMV_TOTAL, :TAX_YEAR, :MKT_YEAR, :TAX_CAPAC, :TOTAL_TAX, :SPEC_ASSES, :USECLASS1, :USECLASS2, :USECLASS3, :USECLASS4, :MULTI_USES, :TAX_EXEMPT, :XUSECLASS1, :XUSECLASS2, :XUSECLASS3, :XUSECLASS4, :DWELL_TYPE, :HOME_STYLE, :FIN_SQ_FT, :GARAGE, :GARAGESQFT, :BASEMENT, :HEATING, :COOLING, :YEAR_BUILT, :NUM_UNITS, :SALE_DATE, :SALE_VALUE, :GREEN_ACRE, :OPEN_SPACE, :AG_PRESERV, :AGPRE_ENRD, :AGPRE_EXPD, :ABB_LEGAL, :EDIT_DATE, :EXP_DATE, :POLYPTRE

In [ ]:
DBFData[[1,10]]

In [12]:
lat, lng = [45.147762, -93.134160]

2-element Array{Float64,1}:
  45.147762
 -93.13416

In [36]:
getDistance(lng, lat, lng, lat)

785441.3358866584

In [105]:
d = []
append!(d, 3)

1-element Array{Any,1}:
 3

In [129]:
function getDistance(locLon, locLat, homeLon, homeLat)
    loc = LLA(locLat, locLon, 0)
    home = LLA(homeLat, homeLon, 0)
    return euclidean_distance(loc, home)
end

function addressSearch(homeLon, homeLat, distance)
    files = glob("../webapp/modules/geotools/geotools/data/new_parcels/*.dbf")
    dfs = []
    for file in files
        DBFData = DBFTables.Table(file)
        distances = DBFData |> TableOperations.map(x-> (Dis=getDistance(Tables.getcolumn(x, :LONGITUDE), Tables.getcolumn(x, :LATITUDE), lng, lat),)) |> Tables.columntable;
        rowTable = DBFData |> Tables.rowtable
        filtered = findall(x -> x <= distance, distances[1])
        df = DataFrame(rowTable[filtered])
        df[!, :Dis] = distances[1][filtered]
#         filter!(row => row[:Dis] <= distance, df)
        push!(dfs, df)
    end
    return vcat(dfs)
end

addressSearch (generic function with 2 methods)

In [130]:
fullDF = addressSearch(lng, lat, 5000)

7-element Array{Any,1}:
 10882×96 DataFrame
   Row │ COUNTY_PIN    STATE_PIN           ANUMBERPRE  ANUMBER  ANUMBERSUF  ST ⋯
       │ String?       String?             String?     Int64?   String?     St ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │ 363123440053  27003-363123440053  missing           0  missing     mi ⋯
     2 │ 363123440052  27003-363123440052  missing           0  missing     mi
     3 │ 363123440051  27003-363123440051  missing         110  missing     mi
     4 │ 363123440050  27003-363123440050  missing         108  missing     mi
     5 │ 363123440049  27003-363123440049  missing         106  missing     mi ⋯
     6 │ 363123440048  27003-363123440048  missing         104  missing     mi
     7 │ 363123440047  27003-363123440047  missing         102  missing     mi
     8 │ 363123440046  27003-363123440046  missing           1  missing     mi
     9 │ 363123440045  27003-363123440045  missing           3  missing     m